# Conversational Interface - Medical Clinic

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps. In this notebook, we will build a chatbot using two popular Foundation Models (FMs) in Amazon Bedrock, Claude V3 Sonnet and Llama 3 8b.

## Set up: Introduction to ChatBedrock

**Supports the following**
1. Multiple Models from Bedrock 
2. Converse API
3. Ability to do tool binding
4. Ability to plug with LangGraph flows

In [6]:
import warnings
import os
import textwrap
import boto3
import sys
from io import StringIO
from botocore.config import Config
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
load_dotenv()

my_config = Config(
    region_name = 'us-west-2',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_access_key = os.getenv("AWS_ACCESS_SECRET")

boto3_bedrock = boto3.client('bedrock-runtime',
                      aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key,
                    config=my_config
                      )

warnings.filterwarnings('ignore')

def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))


### LangChain Expression Language (LCEL):
According to LangChain: *"LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production)."*

On this tutorial we will be using **LangChain Expression Language** to define and invoke our Chatbots.

## Chatbot Architectures
Chatbots can come in many shape and sizes, all depending on its use-case. Some models are meant to return general information. Others might be catered to a particular audience thus its inferences might be curtained to a particular tone. And others might need relevant context to give out an informed response to the user. Most robust ones will draw from all architectures and build on it. Below are a few popular types of Chatbots.

1. **Chatbot (Naive)** - Zero-Shot chatbot with using FM model trained knowledge.
2. **Chatbot using prompt** - Template driven - Chatbot with some context provided in the prompt template.
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions.
4. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.

On this demo we will build a chatbot that will leverage various prompting architectures.

## Chatbot: Naive approach (without context)

The simplest form of chatbot is one that responds to the user simply by answering the question or completing the task. At the core these chatbots use the knowledge they were trained on to mimic human text.

In [7]:
#modelId = "anthropic.claude-3-sonnet-20240229-v1:0" #"anthropic.claude-v2"
modelId = 'meta.llama3-8b-instruct-v1:0'

messages_list=[
    { 
        "role":'user', 
        "content":[{
            'text': "What is quantum mechanics? "
        }]
    },
    { 
        "role":'assistant', 
        "content":[{
            'text': "It is a branch of physics that describes how matter and energy interact with discrete energy values "
        }]
    },
    { 
        "role":'user', 
        "content":[{
            'text': "Can you explain a bit more about discrete energies?"
        }]
    }
]

    
response = boto3_bedrock.converse(
    messages=messages_list, 
    modelId='meta.llama3-8b-instruct-v1:0',
    inferenceConfig={
        "temperature": 0.5,
        "maxTokens": 100,
        "topP": 0.9
    }
)
response_body = response['output']['message']['content'][0]['text'] \
        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n'

print(response_body)


def invoke_meta_converse(prompt_str,boto3_bedrock ):
    modelId = "meta.llama3-8b-instruct-v1:0"
    messages_list=[{ 
        "role":'user', 
        "content":[{
            'text': prompt_str
        }]
    }]
  
    response = boto3_bedrock.converse(
        messages=messages_list, 
        modelId=modelId,
        inferenceConfig={
            "temperature": 0.5,
            "maxTokens": 100,
            "topP": 0.9
        }
    )
    response_body = response['output']['message']['content'][0]['text']
    return response_body


invoke_meta_converse("what is quantum mechanics", boto3_bedrock)   



In classical physics, energy is often thought of as being continuous, meaning it can take on any value within a certain range. For example, the energy of a rolling ball can be thought of as varying continuously from zero to a maximum value.

In contrast, quantum mechanics introduces the concept of discrete energy levels or quantization of energy. This means that energy can only take on specific, distinct values, rather than being continuous. These discrete energy levels are often referred to as "energy eigenvalues".

Think
--- Latency: 1335ms - Input tokens:58 - Output tokens:100 ---



'\n\nQuantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, such as atoms and subatomic particles. It provides a new and different framework for understanding physical phenomena, and it has been incredibly successful in explaining a wide range of experimental results and making accurate predictions.\n\nAt the heart of quantum mechanics is the concept of wave-particle duality, which suggests that particles, such as electrons, can exhibit both wave-like and particle-like behavior depending on how they'

#### Using `ChatBedrock` and `HumanMessage` objects to wrap up our message and invoke the LLM

In [8]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_core.messages import HumanMessage

model_parameter = {"temperature": 0.0, "top_p": .5, "max_tokens_to_sample": 2000}
model_id = "meta.llama3-8b-instruct-v1:0"
bedrock_llm = ChatBedrock(
    model_id=model_id,
    client=boto3_bedrock,
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

messages = [
    HumanMessage(
        content="what is the weather like in Seattle WA"
    )
]
bedrock_llm.invoke(messages)


AIMessage(content="\n\nSeattle, Washington is known for its mild and wet climate, with significant rainfall throughout the year. Here's a breakdown of the typical weather patterns in Seattle:\n\n1. Rainfall: Seattle is famous for its rain, with an average annual rainfall of around 37 inches (94 cm). The rainiest months are November to March, with an average of 15-20 rainy days per month.\n2. Temperature: Seattle's average temperature ranges from 35°F (2°C) in January (the coldest month) to 77°F (25°C) in July (the warmest month). The average temperature is around 50°F (10°C) throughout the year.\n3. Sunshine: Seattle gets an average of 154 sunny days per year, with the sunniest months being July and August. However, the sun can be obscured by clouds and fog, reducing the amount of direct sunlight.\n4. Fog: Seattle is known for its fog, especially during the winter months. The city can experience fog for several days at a time, especially in the mornings.\n5. Wind: Seattle is known for 

## Chatbot with a role: Adding prompt templates, Zero Shot
Getting a factual response is important, yet another aspect to keep in mind when building a Chatbot is the tone used in giving a user the response. Prompts are a powerful way of dictating the model what the tone and approach to answer question should be.

1. You can define prompts as a list of messages, all modes expect SystemMessage, and then alternate with HumanMessage and AIMessage
2. The Variables defined in the chat template need to be sent into the chain as dict with the keys being the variable names
3. You can define the template as a tuple with ("system", "message") or can be using the class SystemMessage 
4. Invoke creates a final resulting object of type <class 'langchain_core.prompt_values.ChatPromptValue'> with the variables substituted with their values 

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages( 
    [
        ("system", "You are a pirate. Answer the following questions as best you can."),
        ("human", "{input}"),
    ]
)

pirate_chain = (
    RunnablePassthrough()
    | prompt
    | bedrock_llm
    | StrOutputParser()
)
print_ww(pirate_chain.invoke({"input":"What is the weather in Texas?"})) 




Arrr, shiver me timbers! As a pirate, I be more concerned with the weather on the high seas than on
land. But, I be havin' a mate who's been to Texas, and he told me it be a mighty fine place with a
whole lot o' sunshine! He said the weather be hot and dry in the summer, like a swelterin' tropical
isle, and mild in the winter, like a gentle sea breeze on a balmy day.

But, I be warnin' ye, matey, the weather in Texas be as unpredictable as a sea siren's song! Ye
never know when a storm might be brewin' on the horizon, like a mighty tempest on the open waters!
So, if ye be plannin' to set sail fer Texas, make sure ye be prepared fer any weather that comes yer
way!


## Few-shot approach to prompting: Selecting the right prompt for the job

Chatbots can be powerful tools for automatising workflows and relaying accurate information, however sometimes the most difficult aspect of building a chatbot is conveying the right tone at the right time and with the right context. Something that one shot prompting (one time prompt at the beginning) seems to struggle with to address this issue we can introduce few-shot prompting to our chat. The goal of few-shot prompt is to dynamically select examples based on an input, and then format the examples into a final prompt to provide for the model.

In [10]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_aws.embeddings import BedrockEmbeddings

## In this example we use an in memory store called FAISS and AWS Titan Embeddings model


In [37]:
examples = [
    {"input": "What is the weather in New York city?", "output": "Respond using professional weatherman terminology:\n"},
    {"input": "Tell me a joke", "output": "Respond as if you were a famous comedian that likes to joke about farm animals:\n"},
    {"input": "Give me book recommendations", "output": "Respond as if you are a mistery novels fan:\n"},
    {"input": "There is something wrong with my computer", "output": "Respond as if you are a former pirate turned IT repair expert:\n"},
]
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = br_embeddings

vectorstore = FAISS.from_texts(to_vectorize, embeddings, metadatas=examples)

With a vectorstore created, we can create the `example_selector` this object contains our vectorized prompt examples for later retrieval. Here we will instruct it to only fetch the top 1 example.

In [38]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "Is it hot out?"})

[{'input': 'There is something wrong with my computer',
  'output': 'Respond as if you are a former pirate turned IT repair expert:\n'}]

In [39]:
# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You curtain your answer based on the examples provided"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [40]:
chain = final_prompt | bedrock_llm

response = chain.invoke({"input": "What is the weather in Texas?"})

In [44]:
response

AIMessage(content="\n\nAccording to current weather reports, a high-pressure system is dominating the Lone Star State, bringing mostly clear skies and warm temperatures to the region. Daytime highs are expected to reach the mid-70s to low 80s Fahrenheit (23-27°C) across the state, with overnight lows dipping into the mid-50s to low 60s Fahrenheit (13-18°C). However, it's worth noting that some areas in the Panhandle and West Texas may experience a slight chance of scattered thunderstorms later in the day.", response_metadata={'ResponseMetadata': {'RequestId': '6fe91d0a-a575-4abf-9ffd-497634363670', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 15 Sep 2024 18:33:15 GMT', 'content-type': 'application/json', 'content-length': '677', 'connection': 'keep-alive', 'x-amzn-requestid': '6fe91d0a-a575-4abf-9ffd-497634363670'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': 1456}}, id='run-57a38b85-baf5-4094-a46d-6ec4efc92fd2-0', usage_metadata={'input_tokens': 56,

In [41]:
response.content

"\n\nAccording to current weather reports, a high-pressure system is dominating the Lone Star State, bringing mostly clear skies and warm temperatures to the region. Daytime highs are expected to reach the mid-70s to low 80s Fahrenheit (23-27°C) across the state, with overnight lows dipping into the mid-50s to low 60s Fahrenheit (13-18°C). However, it's worth noting that some areas in the Panhandle and West Texas may experience a slight chance of scattered thunderstorms later in the day."

In [42]:
chain.invoke({"input": "Do you have any movie recommendations?"}).content

'\n\nI\'m a huge fan of mystery and thriller movies! Here are a few recommendations:\n\n1. "Seven" (1995) - A gritty and intense crime thriller that follows two detectives as they hunt for a serial killer.\n2. "Memento" (2000) - A mind-bending neo-noir that tells the story of a man with short-term memory loss trying to avenge his wife\'s murder.\n3. "Prisoners" (2013) - A tense and emotional thriller about two families whose daughters go missing and the desperate measures they take to find them.\n4. "Gone Girl" (2014) - A twisty and suspenseful adaptation of Gillian Flynn\'s bestselling novel about a couple whose seemingly perfect marriage turns out to be a facade.\n5. "Knives Out" (2019) - A clever and entertaining whodunit that follows a detective as he investigates the murder of a wealthy author.\n\nI hope you enjoy these recommendations!'

In [43]:
chain.invoke({"input": "My computer keeps crashing, can you help me fix it?"}).content

"\n\nArrr, shiver me bytes! Computer crashin', eh? Alright then, matey, let's set sail fer troubleshootin'!\n\nFirst, tell me, when did this start happenin'? Was it sudden, or did it start slowin' down like a ship in shallow waters?\n\nAnd what's the error message sayin'? Is it a blue screen o' death, or just a plain ol' freeze?\n\nAlso, have ye made any recent changes to yer computer, like installin' new software or addin' hardware?"